Разворачиваем и экспериментируем с MongoDB

In [3]:
# для установки используем мануалы с официально сайта
# https://www.mongodb.com/docs/manual/tutorial/install-mongodb-on-ubuntu/

# проверим версию системы
!cat /etc/lsb-release

DISTRIB_ID=Ubuntu
DISTRIB_RELEASE=22.04
DISTRIB_CODENAME=jammy
DISTRIB_DESCRIPTION="Ubuntu 22.04.4 LTS"


In [4]:
# import the MongoDB public GPG key
!curl -fsSL https://www.mongodb.org/static/pgp/server-8.0.asc | \
   sudo gpg -o /usr/share/keyrings/mongodb-server-8.0.gpg \
   --dearmor

In [6]:
# Create the list file for Ubuntu 22.04 (Jammy)
!echo "deb [ arch=amd64,arm64 signed-by=/usr/share/keyrings/mongodb-server-8.0.gpg ] https://repo.mongodb.org/apt/ubuntu jammy/mongodb-org/8.0 multiverse" | sudo tee /etc/apt/sources.list.d/mongodb-org-8.0.list

deb [ arch=amd64,arm64 signed-by=/usr/share/keyrings/mongodb-server-8.0.gpg ] https://repo.mongodb.org/apt/ubuntu jammy/mongodb-org/8.0 multiverse


In [7]:
# Reload the package database
!sudo apt-get update

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://repo.mongodb.org/apt/ubuntu jammy/mongodb-org/8.0 InRelease [3,005 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,383 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 https://repo.mongodb.org/apt/ubuntu jammy/mongodb-org/8.0/multiverse amd64 Packages [28.1 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:12 https://repo.mongodb.org/apt/ubuntu jammy/mongodb-org/8.0/multiverse arm64 Packages [27.

In [8]:
# Install MongoDB Community Server
!sudo apt-get install -y mongodb-org

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  mongodb-database-tools mongodb-mongosh mongodb-org-database
  mongodb-org-database-tools-extra mongodb-org-mongos mongodb-org-server
  mongodb-org-shell mongodb-org-tools
The following NEW packages will be installed:
  mongodb-database-tools mongodb-mongosh mongodb-org mongodb-org-database
  mongodb-org-database-tools-extra mongodb-org-mongos mongodb-org-server
  mongodb-org-shell mongodb-org-tools
0 upgraded, 9 newly installed, 0 to remove and 36 not upgraded.
Need to get 186 MB of archives.
After this operation, 653 MB of additional disk space will be used.
Get:1 https://repo.mongodb.org/apt/ubuntu jammy/mongodb-org/8.0/multiverse amd64 mongodb-database-tools amd64 100.12.0 [58.8 MB]
Get:2 https://repo.mongodb.org/apt/ubuntu jammy/mongodb-org/8.0/multiverse amd64 mongodb-mongosh amd64 2.4.2 [54.5 MB]
Get:3 https://repo.mongodb.org/apt

In [9]:
!mkdir -p /data/db
!sudo chown -R mongodb:mongodb /data/db

In [10]:
!mongod --dbpath /data/db --bind_ip 127.0.0.1 --logpath /data/db/mongod.log --fork

about to fork child process, waiting until server is ready for connections.
forked process: 7084
child process started successfully, parent exiting


In [11]:
# check the server is running
!tail /data/db/mongod.log

{"t":{"$date":"2025-04-08T19:59:56.345+00:00"},"s":"I",  "c":"CONTROL",  "id":20712,   "ctx":"LogicalSessionCacheReap","msg":"Sessions collection is not set up; waiting until next sessions reap interval","attr":{"error":"NamespaceNotFound: config.system.sessions does not exist"}}
{"t":{"$date":"2025-04-08T19:59:56.347+00:00"},"s":"I",  "c":"STORAGE",  "id":20320,   "ctx":"LogicalSessionCacheRefresh","msg":"createCollection","attr":{"namespace":"config.system.sessions","uuidDisposition":"generated","uuid":{"uuid":{"$uuid":"470369f1-0b2d-4016-8e0c-336ca9f32094"}},"options":{}}}
{"t":{"$date":"2025-04-08T19:59:56.392+00:00"},"s":"I",  "c":"INDEX",    "id":20345,   "ctx":"LogicalSessionCacheRefresh","msg":"Index build: done building","attr":{"buildUUID":null,"collectionUUID":{"uuid":{"$uuid":"470369f1-0b2d-4016-8e0c-336ca9f32094"}},"namespace":"config.system.sessions","index":"_id_","ident":"index-5-12646812816752903530","collectionIdent":"collection-4-12646812816752903530","commitTimestam

In [12]:
# install pymongo
!pip install pymongo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 19.4 MB/s eta 0:00:00


In [13]:
from pymongo import MongoClient

# connect to MongoDB
client = MongoClient("mongodb://localhost:27017")
client.list_database_names()

['admin', 'config', 'local']

In [14]:
# create database
db = client['test_database']
collection = db['test_collection']
collection.insert_one({"name": "MongoDB", "type":"Database"})
print("Document inserted:", collection.find_one())

Document inserted: {'_id': ObjectId('67f583c1e47244147e449c83'), 'name': 'MongoDB', 'type': 'Database'}


In [15]:
# find operations
collection = db['search_test_collection']
collection.insert_many([
    {'name': 'Victor', 'age': 45},
    {'name': 'Sasha', 'age': 25},
    {'name': 'Lena', 'age': 35}
])

InsertManyResult([ObjectId('67f586c4e47244147e449c84'), ObjectId('67f586c4e47244147e449c85'), ObjectId('67f586c4e47244147e449c86')], acknowledged=True)

In [16]:
# operation 'equal'
[rec for rec in collection.find({'age':{'$eq':35}})]

[{'_id': ObjectId('67f586c4e47244147e449c86'), 'name': 'Lena', 'age': 35}]

In [17]:
# operation 'not equal'
[rec for rec in collection.find({'age':{'$ne':35}})]

[{'_id': ObjectId('67f586c4e47244147e449c84'), 'name': 'Victor', 'age': 45},
 {'_id': ObjectId('67f586c4e47244147e449c85'), 'name': 'Sasha', 'age': 25}]

In [18]:
# operation 'less than'
[rec for rec in collection.find({'age':{'$lte':35}})]

[{'_id': ObjectId('67f586c4e47244147e449c85'), 'name': 'Sasha', 'age': 25},
 {'_id': ObjectId('67f586c4e47244147e449c86'), 'name': 'Lena', 'age': 35}]

In [19]:
# operation 'in list'
[rec for rec in collection.find({'name':{'$in':['Victor','Lena']}})]

[{'_id': ObjectId('67f586c4e47244147e449c84'), 'name': 'Victor', 'age': 45},
 {'_id': ObjectId('67f586c4e47244147e449c86'), 'name': 'Lena', 'age': 35}]

In [20]:
# operation 'not in list'
[rec for rec in collection.find({'name':{'$nin':['Victor','Lena']}})]

[{'_id': ObjectId('67f586c4e47244147e449c85'), 'name': 'Sasha', 'age': 25}]

In [32]:
# aggregation function
results = collection.aggregate([
    {'$group': {
        '_id': None,
        'age_sum':{'$sum': '$age'},
        'age_avg':{'$avg': '$age'},
        'count':{'$sum': 1}
    }}
])
[rec for rec in results]

[{'_id': None, 'age_sum': 105, 'age_avg': 35.0, 'count': 3}]